In [1]:
import numpy as np
import scipy.optimize as opt
import math
import warnings
warnings.filterwarnings('ignore', 'The iteration is not making good progress')

# Problem 5.1

In [2]:
#Steady-state Equilibrium
#household parameters
beta=0.442
delta=0.6415
sigma=3
#firms parameters
A=1
alpha=0.35
def steadysavings(b2b3vals):
    b2, b3=b2b3vals
    w=(1-alpha)*A*((b2+b3)/2.2)**(alpha)
    r=(alpha*A*(2.2/(b2+b3))**(1-alpha))-delta
    MUc1=(w-b2)**(-sigma)
    MUc2=(w+b2*(1+r)-b3)**(-sigma)
    MUc3=(b3*(1+r)+0.2*w)**(-sigma)
    error1=MUc1-beta*(1+r)*MUc2
    error2=MUc2-beta*(1+r)*MUc3
    errors=np.array([error1,error2])
    return errors
#solve for steady state savings:
b2bar, b3bar=opt.fsolve(steadysavings, (0.01,0.01))
#steady-state wage:
wbar=(1-alpha)*A*((b2bar+b3bar)/2)**(alpha)
#steady-state interest rate:
rbar=(alpha*A*(2/(b2bar+b3bar))**(1-alpha))-delta
#steady-state consumptions:
c1bar=wbar-b2bar
c2bar=wbar+(1+rbar)*b2bar-b3bar
c3bar=0.2*wbar+(1+rbar)*b3bar
print(b2bar,b3bar,wbar,rbar,c1bar,c2bar)

0.0193125298313 0.0584110959209 0.208567399556 2.24836834442 0.189254869724 0.21289051419


# Problem 5.2

In [3]:
#Steady-state Equilibrium
#household parameters
beta=0.55
delta=0.6415
sigma=3
#firms parameters
A=1
alpha=0.35
def steadysavings(b2b3vals):
    b2, b3=b2b3vals
    w=(1-alpha)*A*((b2+b3)/2.2)**(alpha)
    r=(alpha*A*(2.2/(b2+b3))**(1-alpha))-delta
    MUc1=(w-b2)**(-sigma)
    MUc2=(w+b2*(1+r)-b3)**(-sigma)
    MUc3=(b3*(1+r)+0.2*w)**(-sigma)
    error1=MUc1-beta*(1+r)*MUc2
    error2=MUc2-beta*(1+r)*MUc3
    errors=np.array([error1,error2])
    return errors
#solve for steady state savings:
b2bar, b3bar=opt.fsolve(steadysavings, (0.01,0.01))
#steady-state wage:
wbar=(1-alpha)*A*((b2bar+b3bar)/2)**(alpha)
#steady-state interest rate:
rbar=(alpha*A*(2/(b2bar+b3bar))**(1-alpha))-delta
#steady-state consumptions:
c1bar=wbar-b2bar
c2bar=wbar+(1+rbar)*b2bar-b3bar
c3bar=0.2*wbar+(1+rbar)*b3bar
print(b2bar,b3bar,wbar,rbar,c1bar,c2bar,c3bar)


0.0281769188815 0.0768654513042 0.231755707058 1.73452282487 0.203578788177 0.23194068357 0.256541472447


Savings, consumptions and wage go up, while real interest rate goes down. That is because if households are more patient, they are willing to save more (for future use), which results in an increase in capital and thus decrease the real interest rate. 

# Problem 5.3

In [44]:
#Non steady-state equilibrium
Kbar=b2bar+b3bar
#Parameters:
T=30
xi=0.9
tol=1e-9
b21=0.8*b2bar
b31=1.1*b3bar
#guess K:
L=2.2
K=np.ones(T+5)
K[0]=b21+b31
for t in range(T-1,T+5):
    K[t]=Kbar
for i in range(1,T-1):
    K[i]=K[0]+i*(K[T-1]-K[0])/T
#get r from K:
def getr(K):
    r=np.ones(T+5)
    for i in range(0,T+5):
        r[i]=alpha*A*(2.2/K[i])**(1-alpha)-delta
    return r
#get w from K:
def getw(K):
    w=np.ones(T+5)
    for i in range(0,T+5):
        w[i]=(1-alpha)*A*(K[i]/2.2)**(alpha)
    return w
#get K' from K:
def iter(K):     
    r=getr(K)
    w=getw(K)
    b2=np.ones(T+5)
    b3=np.ones(T+5)
    b2[0]=b21
    b3[0]=b31
    #get b32 (middle-aged savings at time 1):
    def getb32(b32):
        return (w[0]+(1+r[0])*b21-b32)**(-sigma)-beta*(1+r[1])*((1+r[1])*b32+0.2*w[1])**(-sigma)
    b3[1]=opt.fsolve(getb32,0.1)[0]
    for t in range(2,T+5):
        def getb2b3(b2b3):
            b2t,b3tplus1=b2b3
            error1=(w[t-2]-b2t)**(-sigma)-beta+(1+r[t-1])*(w[t-1]+(1+r[t-1])*b2t-b3tplus1)**(-sigma)
            error2=(w[t-1]+(1+r[t-1])*b2t-b3tplus1)**(-sigma)-beta*(1+r[t])*((1+r[t])*b3tplus1+0.2*w[t])**(-sigma)
            return ([error1,error2])
        b2[t-1]=opt.fsolve(getb2b3, (0.1,0.1))[0]
        b3[t]=opt.fsolve(getb2b3, (0.1,0.1))[1]
    Knew=np.ones(T+5)
    Knew[0]=K[0]
    for t in range(T-1,T+5):
        Knew[t]=K[t]
    for t in range(1,T-1):
        Knew[t]=b2[t]+b3[t]
    return Knew
#check the norm of K and K':
while np.linalg.norm(K-iter(K))>tol:
    K=xi*iter(K)+(1-xi)*K
#My iteration does not converge and won't run if I include the last part. 

[ 0.10709353  0.12051487  0.11589377  0.11572515  0.11582866  0.11588876
  0.11574907  0.11592892  0.1159002   0.11556411  0.11569163  0.11566418
  0.11535057  0.11551624  0.11561739  0.11582274  0.11567674  0.11562769
  0.11802782  0.16028231  0.16033268  0.16038239  0.16043147  0.16047994
  0.16126596  0.16212805  0.16216939  0.16221028  0.16335709  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11971572  0.11931464  0.12050665  0.11941392  0.11926293
  0.11971872  0.11946531  0.11953861  0.11971072  0.11958626  0.11959196
  0.11981396  0.11972728  0.11967537  0.11972659  0.11980997  0.11985503
  0.12029912  0.11583888  0.12819726  0.13584065  0.13586657  0.13590445
  0.13584917  0.13614136  0.13628773  0.13642955  0.13453484  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11979321  0.11921222  0.12091187  0.12141458  0.12099803
  0.12082958  0.12077668  0.12103979  0.12089397  0.12095744  0.12096089


  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903493  0.1210376   0.12160621  0.12236433
  0.12249801  0.12208294  0.12223332  0.12244243  0.12253787  0.12210127
  0.12240137  0.12220874  0.12232651  0.12256008  0.12271072  0.12273647
  0.12256328  0.1225381   0.1226385   0.12249922  0.12241554  0.12230518
  0.12205207  0.12261384  0.12270227  0.12271642  0.12205364  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972373  0.1191437   0.12105685  0.12163102  0.12181503
  0.12230391  0.12247489  0.1221256   0.12221304  0.1223235   0.12224968
  0.12209504  0.12214411  0.12206499  0.12258153  0.12250449  0.12216563
  0.12241034  0.12247483  0.12249095  0.12229288  0.12215327  0.12199033
  0.12246313  0.12250238  0.12247462  0.12244638  0.1221111   0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11977162  0.11902962  0.12094648  0.12146172  0.12209739
  0.1221456

[ 0.10709353  0.11977162  0.11902962  0.12094659  0.1214733   0.1221067
  0.12199748  0.12231797  0.12241717  0.12254114  0.12209878  0.12248767
  0.12269767  0.12250806  0.12260925  0.12259092  0.12238261  0.12253735
  0.12267133  0.12237276  0.12233849  0.12252563  0.12255357  0.12232359
  0.12256314  0.12248462  0.12261835  0.12221528  0.12190639  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903493  0.12103763  0.12160844  0.12249502
  0.12232707  0.12229401  0.12252539  0.1227156   0.12241007  0.12224965
  0.12221657  0.1225891   0.1223663   0.12232399  0.12235609  0.12255162
  0.1223034   0.12238727  0.12257883  0.12269891  0.12249401  0.12232091
  0.12236712  0.1226683   0.1224699   0.12215827  0.12197771  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972373  0.1191437   0.12105692  0.12163619  0.12229233
  0.12269313  0.12271981  0.12269626  0.12249272  0.12232087  0.12230016
 

  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11977162  0.11902962  0.12094607  0.12146855  0.12214767
  0.12207811  0.12232023  0.1220214   0.12245229  0.12237355  0.12245975
  0.12269596  0.12271549  0.12266966  0.12243152  0.12255067  0.1224824
  0.12222603  0.12235078  0.12236107  0.12209369  0.12231273  0.12213726
  0.12223489  0.12216005  0.12222117  0.12244337  0.12188888  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903492  0.12103751  0.12160901  0.12250791
  0.12236     0.12212324  0.122571    0.1224893   0.12230254  0.12257856
  0.12272136  0.12241331  0.12226791  0.12240377  0.12260254  0.12265238
  0.12267236  0.12226527  0.12258652  0.12234583  0.12204756  0.122512
  0.12250918  0.12231681  0.12226842  0.12238181  0.12178642  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972374  0.11914372  0.121057    0.12163662  0.12229153
  0.12269487  

  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903495  0.12103765  0.12160661  0.12229159
  0.12256355  0.12251878  0.12247111  0.12246318  0.122607    0.12206992
  0.1224977   0.12251246  0.12238232  0.12220756  0.12248008  0.12271881
  0.12270307  0.12272251  0.12273417  0.12271325  0.12241302  0.12231983
  0.12245743  0.12247728  0.12221595  0.12218844  0.12186707  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972373  0.1191437   0.12105685  0.12162827  0.12204359
  0.12200824  0.12255084  0.12248613  0.1222386   0.1223322   0.12242254
  0.1222918   0.1222504   0.12251599  0.12235756  0.12207155  0.12258347
  0.12241506  0.12221528  0.12226474  0.12220481  0.12238787  0.12255858
  0.12269765  0.12247597  0.12242523  0.12226534  0.12199874  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11977162  0.11902962  0.1209464   0.12146831  0.12213959
  0.1224262

...

  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11977161  0.11902963  0.12094613  0.12147203  0.12210309
  0.12222887  0.12227094  0.12229073  0.12267294  0.12249976  0.12252802
  0.12243645  0.12226177  0.12231973  0.12256221  0.12270717  0.12266802
  0.12252115  0.12256746  0.12241741  0.12210536  0.12230032  0.12239175
  0.12222525  0.12221873  0.12255671  0.12248088  0.12195417  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903493  0.12103766  0.12160839  0.12246008
  0.12247318  0.12265148  0.1227352   0.12226765  0.1222851   0.12229883
  0.12252095  0.12250276  0.12255467  0.12271992  0.12272202  0.1221358
  0.12222631  0.122302    0.12231103  0.12220921  0.12215435  0.12265092
  0.1224849   0.12252312  0.12253233  0.12229807  0.12221211  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972374  0.11914368  0.1210569   0.12163469  0.12221734
  0.12257292

  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972373  0.11914374  0.12105697  0.12161676  0.12198142
  0.12211305  0.12224596  0.12244487  0.12257091  0.12242312  0.12253752
  0.12274274  0.12251135  0.12232035  0.122253    0.12247319  0.12270222
  0.12275259  0.12250292  0.1223238   0.12274055  0.1225771   0.12258145
  0.12251337  0.12212697  0.12233904  0.12240287  0.1221259   0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11977161  0.11902963  0.12094608  0.12146747  0.12197872
  0.12214292  0.12225541  0.12188063  0.12260128  0.12261989  0.12274065
  0.12247061  0.12234916  0.12249086  0.12272456  0.12225827  0.12261456
  0.12241899  0.12246177  0.12249217  0.12226382  0.12245196  0.12247445
  0.12265734  0.12248962  0.12210637  0.12266668  0.12211597  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903493  0.12103748  0.12160711  0.12239576
  0.1224862

[ 0.10709353  0.1197716   0.11902964  0.12094665  0.12147371  0.12208271
  0.12219786  0.12227075  0.12271505  0.12234181  0.1223526   0.12242255
  0.1224837   0.12224708  0.12223313  0.12227285  0.12228211  0.12257847
  0.12271941  0.12246993  0.12234852  0.12246735  0.12248251  0.12265858
  0.12249229  0.1222159   0.12213456  0.12226073  0.12209479  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903495  0.12103768  0.12160817  0.12244937
  0.12247696  0.12220869  0.12241029  0.12223704  0.12254665  0.12270832
  0.12271287  0.12265269  0.12270505  0.12271125  0.12273335  0.12227271
  0.12238906  0.1221072   0.12212427  0.1222317   0.12251087  0.12260022
  0.12241261  0.1224827   0.12270823  0.12255575  0.12169667  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11972373  0.11914369  0.12105689  0.12163427  0.12220545
  0.12253075  0.12243729  0.12250094  0.12214716  0.12213504  0.12217268


[ 0.10709353  0.11972373  0.1191437   0.12105686  0.12162916  0.12181869
  0.12230842  0.12271883  0.12273329  0.12265513  0.12241718  0.12255394
  0.12253672  0.12209311  0.12211135  0.12267784  0.12248819  0.12239627
  0.12209353  0.12234348  0.12220288  0.12243602  0.12209473  0.12252325
  0.12264999  0.12272192  0.1222581   0.12231092  0.12214363  0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11977162  0.11902962  0.12094642  0.12146196  0.1221049
  0.12239592  0.12195714  0.12229454  0.12241434  0.12228275  0.12252316
  0.12266932  0.12267072  0.12231909  0.12213185  0.12217324  0.12230778
  0.12229955  0.12210696  0.12252377  0.12251337  0.12211358  0.12215774
  0.12230073  0.12235474  0.12229874  0.12254579  0.1221953   0.10504237
  0.10504237  0.10504237  0.10504237  0.10504237  0.10504237]
[ 0.10709353  0.11954236  0.11903493  0.1210372   0.1216089   0.1224454
  0.12246931  0.1221493   0.12228372  0.12250056  0.12253748  0.12255853
  

KeyboardInterrupt: 